```python
# input_data shape
Input: (batch_size, in_channel, width, height)
# conv layer
class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
input: (Batch_size, C_in, H_in, W_in)
output: (Batch_size, C_out, H_out, W_out)

weight(tensor): (out_channels, in_channels,kernel_size)
bias(tensor): (out_channel)
```

In [1]:
from itertools import product

import time
import numpy as np
from sklearn import datasets
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt

from ripser import Rips
from persim import PersistenceImager

import os
import math

import seaborn
import pandas as pd
import torch
import random
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import persim

import re
import cv2
from PIL import Image

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# parameters
learning_rate = 0.001
training_epochs = 500
batchsize = 25
testbatchsize = 10


In [3]:
# dir = os.listdir("./metric")
# data = []
# a = []
# for metric in dir:
#     a = re.findall("\d+\.?\d*", metric)
#     data.append([np.loadtxt("./metric/"+metric), metric])
#     # print(a[-2])
#     print(metric[-2])


In [4]:
a = np.array([1,2,3])
a

array([1, 2, 3])

In [5]:
class My_dataset(Dataset):
    def __init__(self, train):
        super().__init__()
        dirtrain = os.listdir("./grids_trainset_vgg")
        dirtest = os.listdir("./grids_testset_vgg")
        data = []
        
        if train:
            dir = dirtrain
            path = "./grids_trainset_vgg/"
        else:
            dir = dirtest
            path = "./grids_testset_vgg/"

        T_normal = []
        T_attack = []
        for metric in dir[0:5]:
            dgm = []
            grid = pd.DataFrame(np.loadtxt(path+metric))
            for i in range(grid.shape[0]):
                for j in range(grid.shape[1]):
                    if grid[i][j] > 0:
                        dgm.append([i,j])
            T_normal.append(dgm)
        for metric in dir[-5:]:
            dgm = []
            grid = pd.DataFrame(np.loadtxt(path+metric))
            for i in range(grid.shape[0]):
                for j in range(grid.shape[1]):
                    if grid[i][j] > 0:
                        dgm.append([i,j])
            T_attack.append(dgm)

        print(len(T_attack), len(T_normal))
        
        sample = []
        distance = 1
        for metric in dir:
            sample.append([np.loadtxt(path+metric), metric])
            label = int(metric[0:len("h_vr_metric-single-VGG16_CIFAR=VGG16_CIFAR_attack")]== "h_vr_metric-single-VGG16_CIFAR=VGG16_CIFAR_attack")
            # if (label == 0):
            #     distance_bottleneck, matching = persim.bottleneck(sample[len(sample)-1][0], T_normal[len(sample)-1], matching=True)
            # else:
            #     distance_bottleneck, matching = persim.bottleneck(sample[len(sample)-1][0], T_attack[len(sample)-1], matching=True)
            # distance += distance_bottleneck
            if len(sample) == 5:
                res = cv2.merge([i[0] for i in sample])
                res = np.transpose(res,(2,0,1))
                # data.append([res, np.array((label,1/(1+math.exp(math.log(distance)))))])
                data.append([res, np.array((label,1))])
                sample = []

        self.x = [item[0] for item in data]
        self.y = [item[1] for item in data]
        # self.y = [int(re.findall("\d+\.?\d*", item[1])[0]) for item in data]
        self.src,  self.trg = [], []
        for i in range(len(data)):
            self.src.append(self.x[i])
            self.trg.append(self.y[i])

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)

 # 或者return len(self.trg), src和trg长度一样


In [6]:
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
     
data_train = My_dataset(train=True)
data_test = My_dataset(train=False)
data_loader_train = DataLoader(data_train, batch_size=batchsize, shuffle=True)
data_loader_test = DataLoader(data_test, batch_size=testbatchsize, shuffle=True)


# i_batch的多少根据batch size和def __len__(self)返回的长度确定
# batch_data返回的值根据def __getitem__(self, index)来确定
# 对训练集：(不太清楚enumerate返回什么的时候就多print试试)
print("train")
for i_batch, batch_data in enumerate(data_loader_train):
    print(i_batch)  # 打印batch编号
    print(batch_data[0].shape)  # 打印该batch里面src
    print(batch_data[1])  # 打印该batch里面trg
# # 对测试集：（下面的语句也可以）
print("test")
for i_batch, batch_data in enumerate(data_loader_test):
    print(i_batch)  # 打印batch编号
    print(batch_data[0].shape)  # 打印该batch里面src
    print(batch_data[1])  # 打印该batch里面trg



5 5
5 5
train
0
torch.Size([25, 5, 128, 128])
tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0, 1],
        [0, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [0, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]], dtype=torch.int32)
1
torch.Size([25, 5, 128, 128])
tensor([[0, 1],
        [0, 1],
        [0, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 1],
        [1, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [1, 1],
        [0, 1],
        [0, 1],
        [0, 1]], dtype=torch.int32)
test
0
torch.Size([10, 5, 128, 128])
tensor([[0, 1],
        [0, 1],
        [1, 1

In [13]:
class MyNet(nn.Module):

    def __init__(self):
        super(MyNet,self).__init__()
        self.conv1 = nn.Conv2d(5,64,3,padding=1)
        self.conv2 = nn.Conv2d(64,64,3,padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU()

        self.conv3 = nn.Conv2d(64,128,3,padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3,padding=1)
        self.pool2 = nn.MaxPool2d(2, 2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()

        self.conv5 = nn.Conv2d(128,128, 3,padding=1)
        self.conv6 = nn.Conv2d(128, 128, 3,padding=1)
        self.conv7 = nn.Conv2d(128, 128, 1,padding=1)
        self.pool3 = nn.MaxPool2d(2, 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()

        self.conv8 = nn.Conv2d(128, 256, 3,padding=1)
        self.conv9 = nn.Conv2d(256, 256, 3, padding=1)
        self.conv10 = nn.Conv2d(256, 256, 1, padding=1)
        self.pool4 = nn.MaxPool2d(2, 2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()

        self.conv11 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv12 = nn.Conv2d(512, 512, 3, padding=1)
        self.conv13 = nn.Conv2d(512, 512, 1, padding=1)
        self.pool5 = nn.MaxPool2d(2, 2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()

        self.fc14 = nn.Linear(512*7*7,1024)
        self.drop1 = nn.Dropout2d()
        self.fc15 = nn.Linear(1024,128)
        self.drop2 = nn.Dropout2d()
        self.fc16 = nn.Linear(128,2)


    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.bn1(x)
        x = self.relu1(x)


        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.pool3(x)
        x = self.bn3(x)
        x = self.relu3(x)

        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.pool4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        # print(" x shape ",x.size())
        x = x.view(-1,512*7*7)
        x = F.relu(self.fc14(x))
        x = self.drop1(x)
        x = F.relu(self.fc15(x))
        x = self.drop2(x)
        x = self.fc16(x)

        return x
model = MyNet().to(device)

In [8]:
class CustomLoss2(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    def forward(self, output, target):
        res = -output.gather(dim=1, index=target.view(-1, 1))
        res += torch.log(torch.exp(output).sum(dim=1).view(-1, 1))
        res = res.mean()
        return res
    def forward(self, output, target, distance):
        res = -output.gather(dim=1, index=target.view(-1, 1))
        res += torch.log(torch.exp(output).sum(dim=1).view(-1, 1))
        res = res.mean() * distance.mean()
        return res
        

In [9]:
from typing import Callable, Optional
from torch.nn import functional as F
from torch.nn import _reduction as _Reduction
from torch import Tensor
class _Loss(nn.Module):
    reduction: str

    def __init__(self, size_average=None, reduce=None, reduction: str = 'mean') -> None:
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction: str = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction
class _WeightedLoss(_Loss):
    def __init__(self, weight: Optional[Tensor] = None, size_average=None, reduce=None, reduction: str = 'mean') -> None:
        super(_WeightedLoss, self).__init__(size_average, reduce, reduction)
        self.register_buffer('weight', weight)
        self.weight: Optional[Tensor]
class CELoss(_WeightedLoss): # 注意继承 nn.Module
    __constants__ = ['ignore_index', 'reduction', 'label_smoothing']
    ignore_index: int
    label_smoothing: float

    def __init__(self, weight: Optional[Tensor] = None, size_average=None, ignore_index: int = -100,
                 reduce=None, reduction: str = 'mean', label_smoothing: float = 0.0) -> None:
        super(CELoss, self).__init__(weight, size_average, reduce, reduction)
        self.ignore_index = ignore_index
        self.label_smoothing = label_smoothing

    def forward(self, input: Tensor, target: Tensor, distance) -> Tensor:
        return distance.mean() * F.cross_entropy(input, target, weight=self.weight,
                               ignore_index=self.ignore_index, reduction=self.reduction,
                               label_smoothing=self.label_smoothing)

In [10]:
# define cost/loss & optimizer
# Softmax is internally computed.
# criterion = CELoss().to(device)
criterion = CELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [14]:
total_batch = len(data_loader_train)

for epoch in range(training_epochs):
    model.train()
    avg_cost = 0

    # for X, Y in data_loader_train:
    #     X = torch.autograd.Variable(X).to(device).float()
    #     Y = Y.to(device)

    #     optimizer.zero_grad()
    #     hypothesis = model(X)
    #     cost = criterion(hypothesis, Y)
    #     cost.backward()
    #     optimizer.step()

    #     avg_cost += cost / total_batch
    # model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(data_loader_train):
        distance = targets[:,1].to(device).float()
        targets = targets[:,0]
        inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets, distance)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        avg_cost += loss / total_batch

    
    # if avg_cost < 0.1:
    #     break
    model.eval()
    # with torch.no_grad():
    #     for X_test, Y_test in data_loader_test:
    #         X_test = X_test.to(device).float()
    #         Y_test = Y_test.to(device)

    #     prediction = model(X_test)
    #     correct_prediction = torch.argmax(prediction, 1) == Y_test
    #     accuracy = correct_prediction.float().mean()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_train):
            distance = targets[:,1].to(device).float()
            targets = targets[:,0]
            inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
            outputs = model(inputs)
            loss = criterion(outputs, targets, distance)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    trainacc = 100.*correct/total
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_test):
            distance = targets[:,1].to(device).float()
            targets = targets[:,0]
            inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
            outputs = model(inputs)
            loss = criterion(outputs, targets, distance)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    testacc = 100.*correct/total
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'trainacc:', trainacc, 'testacc:', testacc)

print('Learning finished')


Epoch: 0001 cost = 0.682000160 trainacc: 50.0 testacc: 50.0
Epoch: 0002 cost = 0.706642091 trainacc: 50.0 testacc: 50.0
Epoch: 0003 cost = 0.708194315 trainacc: 50.0 testacc: 50.0
Epoch: 0004 cost = 0.707595527 trainacc: 50.0 testacc: 50.0
Epoch: 0005 cost = 0.693349481 trainacc: 50.0 testacc: 50.0
Epoch: 0006 cost = 0.686443090 trainacc: 50.0 testacc: 50.0
Epoch: 0007 cost = 0.688715816 trainacc: 50.0 testacc: 50.0
Epoch: 0008 cost = 0.692870140 trainacc: 50.0 testacc: 50.0
Epoch: 0009 cost = 0.700643182 trainacc: 50.0 testacc: 50.0
Epoch: 0010 cost = 0.690934777 trainacc: 50.0 testacc: 50.0
Epoch: 0011 cost = 0.685119748 trainacc: 50.0 testacc: 50.0
Epoch: 0012 cost = 0.707437515 trainacc: 50.0 testacc: 50.0
Epoch: 0013 cost = 0.676886797 trainacc: 50.0 testacc: 50.0
Epoch: 0014 cost = 0.709610701 trainacc: 50.0 testacc: 50.0
Epoch: 0015 cost = 0.704070687 trainacc: 50.0 testacc: 50.0
Epoch: 0016 cost = 0.719791889 trainacc: 50.0 testacc: 50.0
Epoch: 0017 cost = 0.708718181 trainacc:

In [12]:
# Test the model using test sets
# model.load_state_dict(torch.load(".\merge_save\merge_attack0normal10_05-07--21-10-32.pth")["state_dict"])
# model.load_state_dict(torch.load(r".\myfed_normal_save\model18-24-02.pth")["state_dict"])
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(data_loader_test):
        distance = targets[:,1].to(device).float()
        targets = targets[:,0]
        inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
        prediction = model(inputs)
        _, predicted = prediction.max(1)

        correct_prediction = torch.argmax(prediction, 1) == targets
        print(prediction,correct_prediction, targets)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    testacc = 100.*correct/total
    print('testacc:', testacc)
    # Get one and predict
    # r = random.randint(0, len(mnist_test) - 1)
    # X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    # Y_single_data = mnist_test.targets[r:r + 1].to(device)

    # print('Label: ', Y_single_data.item())
    # single_prediction = model(X_single_data)
    # print('Prediction: ', torch.argmax(single_prediction, 1).item())


tensor([[ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  1.4875, -3.8505, -3.7793, -3.7843, -3.8735, -3.8153, -3.8244,
         -3.8521, -3.7147],
        [ 1.5041,  